In [130]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from collections import Counter
from operator import itemgetter
import pandas as pd
from sqlalchemy import create_engine

# Calling the nltk package to merge the data of people without existing VIAF URI in the two datasets 
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder

from nltk.metrics import BigramAssocMeasures
from nltk.metrics import TrigramAssocMeasures
import numpy as np


In [131]:
query = """
PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?name ?sName ?uri ?year ?bio
WHERE
  {   { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1770" )
        FILTER ( ( ( ( ( regex(?bio, "juriste", "i") || regex(?bio, "professeur de droit", "i") ) || regex(?bio, "docteur en droit", "i") ) || regex(?bio, "avocat", "i") ) || regex(?bio, "juge", "i") ) || regex(?bio, "magistrat", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf.org", "i")
          }
      }
    UNION
      { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1770" )
        FILTER ( ( ( regex(?bio, "économiste") || regex(?bio, "Economiste") ) || regex(?bio, "professeur d'économie", "i") ) || regex(?bio, "docteur en économie", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri1
            FILTER regex(?uri1, "viaf.org", "i")
            BIND(strbefore(str(?uri1), "http://viaf.org/viaf/") AS ?uri)
          }
      }
  }
ORDER BY DESC(?uri)


"""

In [132]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql") ##, returnFormat=RDFXML)  [LOCALHOST]

In [133]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [134]:
try:
    rc_bnf = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [135]:
# Number of rows in the result
len(rc_bnf['results']['bindings'])

11201

In [136]:
# Inspect the first three rows
i = 0
for l in rc_bnf['results']['bindings']:
    if i < 100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb12981404c#about'}, 'name': {'type': 'literal', 'value': 'Léon Garnier'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99996033'}, 'year': {'type': 'literal', 'value': '1836'}, 'bio': {'type': 'literal', 'value': "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"}}
{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb13484444m#about'}, 'name': {'type': 'literal', 'value': 'Gaston de Pawlowski'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9999219'}, 'year': {'type': 'literal', 'value': '1874'}, 'bio': {'type': 'literal', 'value': 'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"'}}
{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb134841632#about'}, 'name': {'type': 'literal', 'value': 'Jean-Michel Berton'}, 'uri': {'type': 'uri', 'value': 'http://viaf.o

In [137]:
result_bnf = []
for l in rc_bnf['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            sName = l['sName']['value']
        except Exception as e:
            sName = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            bio = l['bio']['value']
        except Exception as e:
            bio = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            year = ''
        result_bnf.append([l['s']['value'], uri, name, sName, year, bio])        
            
        

In [138]:
print(len(result_bnf))
result_bnf[:2]

11201


[['http://data.bnf.fr/ark:/12148/cb12981404c#about',
  'http://viaf.org/viaf/99996033',
  'Léon Garnier',
  '',
  '1836',
  "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"],
 ['http://data.bnf.fr/ark:/12148/cb13484444m#about',
  'http://viaf.org/viaf/9999219',
  'Gaston de Pawlowski',
  '',
  '1874',
  'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"']]

In [139]:
query_2= """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?uri ?name (year(xsd:dateTime(?Birth_Date)) AS ?year) ?abstract
WHERE
  {   { ?s  a              dbo:Economist ;
          # "Economist" has a class function, he explains the use of the "a".
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
          # We use this filter to preserve only the persons born after 1800. 
          # Here, the method is a little different because we have a date and not a year. 
          # We convert above the date to a year to have the same format as BnF Data.
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name }
        FILTER ( xsd:string(?name) != "Samuel Bowles" )
     # We use this filter to remove the data concerning "Samuel Bowles" 
     # because there is mistakes in data i.e he is related to wrong people -eg. William Turner (cf. https://dbpedia.org/page/Samuel_Bowles_(economist)).
     # We will can fix this problem later, by entering data about him manually.
      }
    UNION
      { ?s  ?propriety  dbr:Economist
      # Here, "Economist" is a instance, we have as triplet "subject predicate object". 
      # In DBpedia, "Economist" is define as a instance and at the same time as a class. So we use both.
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }    
      }
    UNION
      { ?s  ?p             dbr:Jurist ;
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
       # We use this filter to remove the data concerning "Cicero". 
       # He appears in results, certainly because his date of birth contains the sign "-" (cf. https://dbpedia.org/page/Cicero).
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
      }
    UNION
      { ?s  ?p             dbr:Lawyer ;
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
      }
    UNION
      { ?s  a              dbr:Professor ;
            dbp:birthDate  ?Birth_Date ;
            dbo:abstract   ?abstract
        FILTER ( ( ( ( regex(?abstract, "lawyer", "i") || regex(?abstract, "jurist", "i") ) || regex(?abstract, "juriste", "i") ) || regex(?abstract, "attorney", "i") ) || regex(?abstract, "legal professional", "i") )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
      }
  }
ORDER BY DESC(?uri)
"""

In this query, we have made the choice to aggregate, by a UNION clause, several queries to maximise the results' number. Also we request the "economists" and the "jurists" in only one query. 

Obviously, we chose classes and instances directly related to our population, but also the "professor" instance, because some "economists" or "jurists" are in this instance (we have tried with and without them, and there more result when we use them). 

Also, we excluse all classes because they don't add more result, except the "Economist" class (we keep it) 

For exemple, we exclude the resource "personFunction" and the resource "Jurists" because they add no more data. Additionally, we keep only the "Professor" instance for the jurists (it returns result only for the jurists).

In [140]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [141]:
sparql.setQuery(query_2)
sparql.setReturnFormat(JSON)

In [142]:
rc_db = sparql.queryAndConvert()

In [143]:
# Number of rows in the result
len(rc_db['results']['bindings'])

8545

In [144]:
# Inspect the first three rows
i = 0
for l in rc_db['results']['bindings']:
    if i <100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/António_de_Almeida_Santos'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99921066'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'António de Almeida Santos'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1926'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Anita_Augspurg'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9976800'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Anita Augspurg'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1857'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Mason_Gaffney'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9960617'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Mason Gaffney'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1923'}}
{'s'

In [145]:
# Create a list with URI, VIAF URI, name, year
result_dbpedia = []
for l in rc_db['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            uri = ''
        result_dbpedia.append([l['s']['value'], uri, name, year])

In [146]:
# Inspect the first three of the list
result_dbpedia[:10]

[['http://dbpedia.org/resource/António_de_Almeida_Santos',
  'http://viaf.org/viaf/99921066',
  'António de Almeida Santos',
  '1926'],
 ['http://dbpedia.org/resource/Anita_Augspurg',
  'http://viaf.org/viaf/9976800',
  'Anita Augspurg',
  '1857'],
 ['http://dbpedia.org/resource/Mason_Gaffney',
  'http://viaf.org/viaf/9960617',
  'Mason Gaffney',
  '1923'],
 ['http://dbpedia.org/resource/Hermann_Heinrich_Gossen',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '1810'],
 ['http://dbpedia.org/resource/Gottfried_Haberler',
  'http://viaf.org/viaf/99257315',
  'Gottfried Haberler',
  '1900'],
 ['http://dbpedia.org/resource/Michael_C._Burda',
  'http://viaf.org/viaf/9922987',
  'Michael C. Burda',
  '1959'],
 ['http://dbpedia.org/resource/Xavier_Vives',
  'http://viaf.org/viaf/9920331',
  'Xavier Vives',
  '1955'],
 ['http://dbpedia.org/resource/Vittorio_Emanuele_Orlando',
  'http://viaf.org/viaf/9914155',
  'Vittorio Emanuele Orlando',
  '1860'],
 ['http://dbpedia.org/reso

In [147]:
query_3= """
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  wd:   <http://www.wikidata.org/entity/>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT  ?s ?uri ?name ?year
WHERE
  {   { ?s  wdt:P106  wd:Q188094 }
    UNION
      { ?s  wdt:P106  wd:Q185351 }
    UNION
      { ?s  wdt:106  wd:Q40348 }
    OPTIONAL
      { ?s  wdt:P570  ?dod }
    OPTIONAL
      { ?s  wdt:P569  ?dob }
    OPTIONAL
      { ?s  wdt:P214  ?oldURI
        BIND(uri(concat("http://viaf.org/viaf/", strafter(str(?oldURI), ""))) AS ?uri) # It's useful to have the URI VIAF in the same and merge data.
      }
    BIND(year(?dob) AS ?year)
    FILTER ( ?year > 1770 )
    OPTIONAL
      { ?s  rdfs:label  ?name
        FILTER ( lang(?name) = "en" )
      }
  }
"""

In [148]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [149]:
sparql.setQuery(query_3)
sparql.setReturnFormat(JSON)

In [150]:
rc_wk = sparql.queryAndConvert()

In [151]:
# Number of rows in the result
len(rc_wk['results']['bindings'])

53434

In [152]:
# Create a list with URI, VIAF URI, name, year
result_wikidata = []
for l in rc_wk['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            uri = ''
        result_wikidata.append([l['s']['value'], uri, name, year])

In [153]:
result_wikidata[:10]

[['http://www.wikidata.org/entity/Q65587',
  'http://viaf.org/viaf/44344307',
  'Werner Maihofer',
  '1918'],
 ['http://www.wikidata.org/entity/Q65539',
  'http://viaf.org/viaf/4137633',
  'Peter Altmaier',
  '1958'],
 ['http://www.wikidata.org/entity/Q65561',
  'http://viaf.org/viaf/232142151',
  'Hans Apel',
  '1932'],
 ['http://www.wikidata.org/entity/Q85944',
  'http://viaf.org/viaf/3369267',
  'Christoph Flügge',
  '1947'],
 ['http://www.wikidata.org/entity/Q86043',
  'http://viaf.org/viaf/27864307',
  'Hugo Sinzheimer',
  '1875'],
 ['http://www.wikidata.org/entity/Q85917',
  'http://viaf.org/viaf/786495',
  'Adolf Wermuth',
  '1855'],
 ['http://www.wikidata.org/entity/Q85842',
  'http://viaf.org/viaf/54138061',
  'Leopold August Warnkönig',
  '1794'],
 ['http://www.wikidata.org/entity/Q85844',
  'http://viaf.org/viaf/8275505',
  'Wolfgang Bötsch',
  '1938'],
 ['http://www.wikidata.org/entity/Q85899',
  'http://viaf.org/viaf/15569664',
  'Hans Hermann von Berlepsch',
  '1843'],
 [

In [172]:
engine = create_engine('sqlite:///database.sqlite', echo=False)

In [173]:
df_bnf = pd.DataFrame([f for f in result_bnf if len(f[1]) > 0], columns=['uri_bnf', 'viaf', 'name_bnf', 'sName', 'year_bnf', 'bio_bnf'])
print(len(df_bnf))
df_bnf.fillna('')

df_bnf.head()

9147


,uri_bnf,viaf,name_bnf,sName,year_bnf,bio_bnf
0,http://data.bnf.fr/ark:/12148/cb12981404c#about,http://viaf.org/viaf/99996033,Léon Garnier,,1836,Juriste. - Administrateur et homme de lettres....
1,http://data.bnf.fr/ark:/12148/cb13484444m#about,http://viaf.org/viaf/9999219,Gaston de Pawlowski,,1874,Docteur en droit. - Critique littéraire et thé...
2,http://data.bnf.fr/ark:/12148/cb134841632#about,http://viaf.org/viaf/9999131,Jean-Michel Berton,,1794,"Écrivain politique, avocat à la Cour de cassat..."
3,http://data.bnf.fr/ark:/12148/cb13379520q#about,http://viaf.org/viaf/9995247,Emmanuel Mathieu,,1852,"Docteur en droit (Paris, 1873)"
4,http://data.bnf.fr/ark:/12148/cb13338312g#about,http://viaf.org/viaf/9994322,Josiah Henry Benton,,1843,Juriste. - Bibliophile


In [174]:
df_bnf.to_sql('df_bnf', con=engine)

In [175]:
df_dbp = pd.DataFrame([p for p in result_dbpedia if len(p[1]) > 0], columns=['uri_dbp', 'viaf', 'name_dbp', 'date_dbp'])
print(len(df_dbp))
df_dbp.fillna('')
df_dbp[-10:]

1747


,uri_dbp,viaf,name_dbp,date_dbp
1737,http://dbpedia.org/resource/Paul_A._Baran,http://viaf.org/viaf/100304945,Paul Alexander Baran,1909
1738,http://dbpedia.org/resource/Menachem_Elon,http://viaf.org/viaf/100286138,Menachem Elon,1923
1739,http://dbpedia.org/resource/József_Antall,http://viaf.org/viaf/100281452,József Antall,1932
1740,http://dbpedia.org/resource/Luiz_Carlos_Bresse...,http://viaf.org/viaf/100278275,Luiz Carlos Bresser-Pereira,1934
1741,http://dbpedia.org/resource/Michael_Kaser,http://viaf.org/viaf/100274542,Michael Kaser,1926
1742,http://dbpedia.org/resource/Merton_Miller,http://viaf.org/viaf/100258394,Merton Miller,1923
1743,http://dbpedia.org/resource/Peter_J._Hammond_(...,http://viaf.org/viaf/100246974,Peter Hammond,1945
1744,http://dbpedia.org/resource/José_Hermano_Saraiva,http://viaf.org/viaf/100238197,José Hermano Saraiva,1919
1745,http://dbpedia.org/resource/Alf_Ross,http://viaf.org/viaf/100233686,Alf Ross,1899
1746,http://dbpedia.org/resource/Max_Weber,http://viaf.org/viaf/100180950,,1864


In [176]:
df_dbp.to_sql('df_dbp', con=engine)

In [177]:
df_wk= pd.DataFrame([p for p in result_wikidata if len(p[1]) > 0], columns=['uri_wk', 'viaf', 'name_wk', 'date_wk'])
print(len(df_wk))
df_wk.fillna('')
df_wk.head()

40358


,uri_wk,viaf,name_wk,date_wk
0,http://www.wikidata.org/entity/Q65587,http://viaf.org/viaf/44344307,Werner Maihofer,1918
1,http://www.wikidata.org/entity/Q65539,http://viaf.org/viaf/4137633,Peter Altmaier,1958
2,http://www.wikidata.org/entity/Q65561,http://viaf.org/viaf/232142151,Hans Apel,1932
3,http://www.wikidata.org/entity/Q85944,http://viaf.org/viaf/3369267,Christoph Flügge,1947
4,http://www.wikidata.org/entity/Q86043,http://viaf.org/viaf/27864307,Hugo Sinzheimer,1875


In [178]:
df_wk.to_sql('df_wk', con=engine)

In [192]:
merged_df_1 = pd.merge( df_wk, df_dbp , on='viaf', how='inner', sort='date_dbp')

In [202]:
print("proportion of the number of merged data from DBpedia with Wikidata:")
print("")
print((len(merged_df_1))/(len(df_dbp))*100)
print("")
print("proportion of the number of merged data from Wikidata with Wikidata")
print("")
print((len(merged_df_1))/(len(df_wk))*100)

proportion of the number of merged data from DBpedia with Wikidata

66.05609616485404

proportion of the number of merged data from Wikidata with Wikidata

2.8594082957530107


In [203]:
merged_df_1[:5]

,uri_wk,viaf,name_wk,date_wk,uri_dbp,name_dbp,date_dbp
0,http://www.wikidata.org/entity/Q9387,http://viaf.org/viaf/100180950,Max Weber,1864,http://dbpedia.org/resource/Max_Weber,,1864
1,http://www.wikidata.org/entity/Q510386,http://viaf.org/viaf/100238197,José Hermano Saraiva,1919,http://dbpedia.org/resource/José_Hermano_Saraiva,José Hermano Saraiva,1919
2,http://www.wikidata.org/entity/Q15999850,http://viaf.org/viaf/100246974,Peter J. Hammond,1945,http://dbpedia.org/resource/Peter_J._Hammond_(...,Peter Hammond,1945
3,http://www.wikidata.org/entity/Q157255,http://viaf.org/viaf/100258394,Merton Miller,1923,http://dbpedia.org/resource/Merton_Miller,Merton Miller,1923
4,http://www.wikidata.org/entity/Q15990097,http://viaf.org/viaf/100274542,Michael Kaser,1926,http://dbpedia.org/resource/Michael_Kaser,Michael Kaser,1926


In [189]:
merged_df = pd.merge( merged_df_1, df_dbp , on='viaf', how='inner', sort='viaf')

In [205]:
print((len(merged_df))/(len(df_dbp))*100)

2.919290211791643


In [191]:
merged_df

,uri_wk,viaf,name_wk,date_wk,uri_bnf,name_bnf,sName,year_bnf,bio_bnf,uri_dbp,name_dbp,date_dbp
0,http://www.wikidata.org/entity/Q437403,http://viaf.org/viaf/100966624,John Peters Humphrey,1905,http://data.bnf.fr/ark:/12148/cb122145877#about,John Humphrey,,1905,Juriste. - A été professeur de droit internati...,http://dbpedia.org/resource/John_Peters_Humphrey,John Peters Humphrey,1905
1,http://www.wikidata.org/entity/Q230814,http://viaf.org/viaf/107536763,Louis Renault,1843,http://data.bnf.fr/ark:/12148/cb12327654n#about,Louis Renault,,1843,Juriste. - Professeur de droit international à...,http://dbpedia.org/resource/Louis_Renault_(jur...,Louis Renault,1843
2,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,...",http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922
3,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938
4,http://www.wikidata.org/entity/Q1730876,http://viaf.org/viaf/108794549,Karl Engisch,1899,http://data.bnf.fr/ark:/12148/cb12299375j#about,Karl Engisch,,1899,Juriste. - Spécialiste de philosophie du droit...,http://dbpedia.org/resource/Karl_Engisch,Karl Engisch,1899
5,http://www.wikidata.org/entity/Q5258787,http://viaf.org/viaf/19719687,Tobias Barreto,1839,http://data.bnf.fr/ark:/12148/cb121737508#about,Tobias Barreto,,1839,"Philosophe, juriste. - Poète, essayiste",http://dbpedia.org/resource/Tobias_Barreto,Tobias Barreto,1839
6,http://www.wikidata.org/entity/Q942703,http://viaf.org/viaf/204968982,Alf Ross,1899,http://data.bnf.fr/ark:/12148/cb129377723#about,Alf Ross,,1899,"Juriste, spécialisé en philosophie du droit, d...",http://dbpedia.org/resource/Alf_Ross,Alf Ross,1899
7,http://www.wikidata.org/entity/Q3619732,http://viaf.org/viaf/20961289,Antonio Mario La Pergola,1931,http://data.bnf.fr/ark:/12148/cb12343730c#about,Antonio La Pergola,,1931,Juriste. - Président de la Cour constitutionnelle,http://dbpedia.org/resource/Antonio_Mario_La_P...,Antonio Mario La Pergola,1931
8,http://www.wikidata.org/entity/Q90243,http://viaf.org/viaf/22222311,Otto Lenel,1849,http://data.bnf.fr/ark:/12148/cb12380018p#about,Otto Lenel,,1849,Juriste. - Spécialiste de droit romain. - Memb...,http://dbpedia.org/resource/Otto_Lenel,Otto Lene,1849
9,http://www.wikidata.org/entity/Q216872,http://viaf.org/viaf/24716169,Tobias Asser,1838,http://data.bnf.fr/ark:/12148/cb12551967b#about,Tobias-Michael-Carel Asser,,1838,Juriste. - Fut l'un des fondateurs de l'Instit...,http://dbpedia.org/resource/Tobias_Asser,Tobias Asser,1838


In [207]:
# Append DBpedia and BnF Data to the merged Dataframe who don't have viaf.org values in common.

# To achieve this, I am served on these pages: 
# https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html (en)
# https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html (en)
# http://www.python-simple.com/python-pandas/concatenations-joins-dataframe.php (fr)

result = merged_df.append([df_dbp, df_bnf, df_wk], sort=False)
print(len(result))
result[:150]

51303


,uri_wk,viaf,name_wk,date_wk,uri_bnf,name_bnf,sName,year_bnf,bio_bnf,uri_dbp,name_dbp,date_dbp
0,http://www.wikidata.org/entity/Q437403,http://viaf.org/viaf/100966624,John Peters Humphrey,1905,http://data.bnf.fr/ark:/12148/cb122145877#about,John Humphrey,,1905,Juriste. - A été professeur de droit internati...,http://dbpedia.org/resource/John_Peters_Humphrey,John Peters Humphrey,1905
1,http://www.wikidata.org/entity/Q230814,http://viaf.org/viaf/107536763,Louis Renault,1843,http://data.bnf.fr/ark:/12148/cb12327654n#about,Louis Renault,,1843,Juriste. - Professeur de droit international à...,http://dbpedia.org/resource/Louis_Renault_(jur...,Louis Renault,1843
2,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,...",http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922
3,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938
4,http://www.wikidata.org/entity/Q1730876,http://viaf.org/viaf/108794549,Karl Engisch,1899,http://data.bnf.fr/ark:/12148/cb12299375j#about,Karl Engisch,,1899,Juriste. - Spécialiste de philosophie du droit...,http://dbpedia.org/resource/Karl_Engisch,Karl Engisch,1899
5,http://www.wikidata.org/entity/Q5258787,http://viaf.org/viaf/19719687,Tobias Barreto,1839,http://data.bnf.fr/ark:/12148/cb121737508#about,Tobias Barreto,,1839,"Philosophe, juriste. - Poète, essayiste",http://dbpedia.org/resource/Tobias_Barreto,Tobias Barreto,1839
6,http://www.wikidata.org/entity/Q942703,http://viaf.org/viaf/204968982,Alf Ross,1899,http://data.bnf.fr/ark:/12148/cb129377723#about,Alf Ross,,1899,"Juriste, spécialisé en philosophie du droit, d...",http://dbpedia.org/resource/Alf_Ross,Alf Ross,1899
7,http://www.wikidata.org/entity/Q3619732,http://viaf.org/viaf/20961289,Antonio Mario La Pergola,1931,http://data.bnf.fr/ark:/12148/cb12343730c#about,Antonio La Pergola,,1931,Juriste. - Président de la Cour constitutionnelle,http://dbpedia.org/resource/Antonio_Mario_La_P...,Antonio Mario La Pergola,1931
8,http://www.wikidata.org/entity/Q90243,http://viaf.org/viaf/22222311,Otto Lenel,1849,http://data.bnf.fr/ark:/12148/cb12380018p#about,Otto Lenel,,1849,Juriste. - Spécialiste de droit romain. - Memb...,http://dbpedia.org/resource/Otto_Lenel,Otto Lene,1849
9,http://www.wikidata.org/entity/Q216872,http://viaf.org/viaf/24716169,Tobias Asser,1838,http://data.bnf.fr/ark:/12148/cb12551967b#about,Tobias-Michael-Carel Asser,,1838,Juriste. - Fut l'un des fondateurs de l'Instit...,http://dbpedia.org/resource/Tobias_Asser,Tobias Asser,1838


In [175]:
# Replace each null value with the value from the other database.
# Create a column with the fresh values.
# I used this page: https://kanoki.org/2019/08/17/pandas-coalesce-replace-value-from-another-column/ 
# particuliarly, the "Using Numpy" section

result_test=result
result_test['name'] = np.where(result['name_bnf'].isnull(),result['name_dbp'],result['name_bnf'])
result_test['name'] = np.where(result['name_dbp'].isnull(),result['name_bnf'],result['name_dbp'])
result_test['year'] = np.where(result['date_dbp'].isnull(),result['year_bnf'],result['date_dbp'])
result_test['year'] = np.where(result['year_bnf'].isnull(),result['date_dbp'],result['year_bnf'])


result_test=result_test.loc[:,['viaf','name','year','uri_dbp','uri_bnf']]
result_test

,viaf,name,year,uri_dbp,uri_bnf
0,http://viaf.org/viaf/100966624,John Peters Humphrey,1905,http://dbpedia.org/resource/John_Peters_Humphrey,http://data.bnf.fr/ark:/12148/cb122145877#about
1,http://viaf.org/viaf/107536763,Louis Renault,1843,http://dbpedia.org/resource/Louis_Renault_(jur...,http://data.bnf.fr/ark:/12148/cb12327654n#about
2,http://viaf.org/viaf/108173876,,1931,http://dbpedia.org/resource/Ronald_Dworkin,http://data.bnf.fr/ark:/12148/cb122775427#about
3,http://viaf.org/viaf/108565309,Paul Abraham Freund,1908,http://dbpedia.org/resource/Paul_A._Freund,http://data.bnf.fr/ark:/12148/cb120906270#about
4,http://viaf.org/viaf/108624624,CH Muhammad Zafarullah Khan,1893,http://dbpedia.org/resource/Muhammad_Zafarulla...,http://data.bnf.fr/ark:/12148/cb128832222#about
5,http://viaf.org/viaf/108794549,Karl Engisch,1899,http://dbpedia.org/resource/Karl_Engisch,http://data.bnf.fr/ark:/12148/cb12299375j#about
6,http://viaf.org/viaf/111389197,Georges Hilaire Bousquet,1846,http://dbpedia.org/resource/Georges_Hilaire_Bo...,http://data.bnf.fr/ark:/12148/cb118935370#about
7,http://viaf.org/viaf/11396531,John Paul Stevens,1920,http://dbpedia.org/resource/John_Paul_Stevens,http://data.bnf.fr/ark:/12148/cb12328362p#about
8,http://viaf.org/viaf/114273114,Ernesto Dihigo,1896,http://dbpedia.org/resource/Ernesto_Dihigo,http://data.bnf.fr/ark:/12148/cb12264150t#about
9,http://viaf.org/viaf/13104809,,1802,http://dbpedia.org/resource/Julius_von_Kirchmann,http://data.bnf.fr/ark:/12148/cb106607262#about


In [176]:
# Extract only rows without VIAF uri from DBpeida
df_dbp_test = pd.DataFrame(result_dbpedia, columns=['uri_dbp', 'viaf', 'name_dbp', 'date_dbp'])

df_dbp_test_mask=df_dbp_test['viaf']==''
filtered_df_dbp_test = df_dbp_test[df_dbp_test_mask]
filtered_df_dbp_test

,uri_dbp,viaf,name_dbp,date_dbp
731,http://dbpedia.org/resource/Luis_Villares,,Luis Villares,1978
732,http://dbpedia.org/resource/Madzy_Rollin_Couqu...,,Madzy Rollin Couquerque,1903
733,http://dbpedia.org/resource/Mahmud_Mohammed,,Mahmud Mohammed,1946
734,http://dbpedia.org/resource/V._S._Malimath,,V. S. Malimath,1929
735,http://dbpedia.org/resource/Valery_Falkov,,Valery Falkov,1978
736,http://dbpedia.org/resource/Verena_Madner,,Verena Madner,1965
737,http://dbpedia.org/resource/Syed_Wazir_Hasan,,Syed Wazir Hasan,1874
738,http://dbpedia.org/resource/Blandine_Brocard,,Blandine Brocard,1981
739,http://dbpedia.org/resource/Bode_Rhodes-Vivour,,Bode Rhodes-Vivour,1951
740,http://dbpedia.org/resource/Dionysius_Koolen,,Dionysius Koolen,1871


In [177]:
# Extract only rows without VIAF uri from BnF Data
df_bnf_test = pd.DataFrame(result_bnf, columns=['uri_bnf', 'viaf', 'name_bnf', 'sName','year_bnf','bio_bnf'])

df_bnf_test_mask=df_bnf_test['viaf']==''
filtered_df_bnf_test = df_bnf_test[df_bnf_test_mask]
len(filtered_df_bnf_test)

1336

In [178]:
# merge the both dateframes 
filtered_dbp_bnf_test= filtered_df_dbp_test.append(filtered_df_bnf_test, sort=True)
len(filtered_dbp_bnf_test)

7799

In [263]:
# Replace each null value with the value from the other database.
# Create a column with the fresh values.
# I used this page: https://kanoki.org/2019/08/17/pandas-coalesce-replace-value-from-another-column/ 
# particuliarly, the "Using Numpy" section

result_f_test=filtered_dbp_bnf_test
result_f_test['name'] = np.where(filtered_dbp_bnf_test['name_bnf'].isnull(),filtered_dbp_bnf_test['name_dbp'],filtered_dbp_bnf_test['name_bnf'])
result_f_test['name'] = np.where(filtered_dbp_bnf_test['name_dbp'].isnull(),filtered_dbp_bnf_test['name_bnf'],filtered_dbp_bnf_test['name_dbp'])
result_f_test['year'] = np.where(filtered_dbp_bnf_test['date_dbp'].isnull(),filtered_dbp_bnf_test['year_bnf'],filtered_dbp_bnf_test['date_dbp'])
result_f_test['year'] = np.where(filtered_dbp_bnf_test['year_bnf'].isnull(),filtered_dbp_bnf_test['date_dbp'],filtered_dbp_bnf_test['year_bnf'])


result_f_test=result_f_test.loc[:,['name','year','uri_dbp','uri_bnf']]
sort_rft=result_f_test.sort_values(by='name', ascending=False)
sort_rft

,name,year,uri_dbp,uri_bnf
1150,วิษณุ เครืองาม,1951,http://dbpedia.org/resource/Wissanu_Krea-ngam,NaN
1058,ناصر یگانه,1921,http://dbpedia.org/resource/Nasser_Yeganeh,NaN
895,آیت اللہ حسن رضا غدیری,1952,http://dbpedia.org/resource/Hassan_Raza_Ghadeeri,NaN
772,Чавдар Кюранов,1921,http://dbpedia.org/resource/Chavdar_Kyuranov,NaN
1026,Тамас Айтмұхамбетов,1939,http://dbpedia.org/resource/Tamas_Aitmukhambetov,NaN
1107,Мұсабек Әлімбеков,1957,http://dbpedia.org/resource/Musabek_Alimbekov,NaN
842,Михаил Фёдорович Малахов,1946,http://dbpedia.org/resource/Mikhail_Fedorovich...,NaN
1161,Бектас Бекназаров,1956,http://dbpedia.org/resource/Bektas_Beknazarov,NaN
10657,̓Ioánnīs ̓Alexándros Meletopoúlos,1903,NaN,http://data.bnf.fr/ark:/12148/cb11573419v#about
10047,̓Euággelos Kampéros,1920,NaN,http://data.bnf.fr/ark:/12148/cb17873286s#about


In [270]:
length= len(sort_rft)
sort_rft.loc[1400]=[sort_rft["name"][440],
                  sort_rft["year"][440],
                  sort_rft["uri_dbp"][440],
                  sort_rft["uri_bnf"][440]]

KeyError: 440

In [185]:
# Test to find similarities between the names BnF Data and DBpedia with collocation 


from nltk.corpus import stopwords
stopset = stopwords.words('english')
stopset = stopwords.words('french')
stopset = stopwords.words('russian')

result_f_test['tokenized_sents'] = result_f_test.apply(lambda row: nltk.word_tokenize(row['name']), axis=1)

bcf = TrigramCollocationFinder.from_documents(result_f_test['tokenized_sents'])
filter_stops = lambda w: w in stopset
bcf.apply_word_filter(filter_stops)
f= bcf.nbest(TrigramAssocMeasures., 50)
print(f)

[('Farkas', 'de', 'Boldogfa'), ('Faton', 'de', 'Favernay'), ('Hari', 'Singh', 'Gour'), ('Hassan', 'Raza', 'Ghadeeri'), ('Henríquez', 'y', 'Carvajal'), ('Jiménez', 'de', 'Aréchaga'), ("'", 'Antoine', 'de'), ("'", 'Argent', 'de'), ('(', 'Сергій', 'Льовочкін'), ('(', 'سعید', 'الزمان'), ('(', 'شیخ', 'انوار'), (',', 'Annie', 'Pommeray'), (',', 'Horace', 'Worth'), (',', 'Josué', 'Crémieux'), (',', 'Louis', ','), (',', 'Pierre', 'Gendre'), (',', 'Sr', '.'), ('A.', 'Boyd', 'Jr'), ('Abang', 'Iskandar', 'Abang'), ('Abbās', 'ibn', 'Ibrāhīm'), ('Adish', 'Chandra', 'Aggarwala'), ('Adnan', 'Iqbal', 'Chaudhry'), ('Adolphe', 'Leroy', 'de'), ('Adrien', 'de', 'Tourville'), ('Adrien', 'de', 'Valon'), ('Afonso', 'Pena', 'Júnior'), ('Agustín', 'Edwards', 'McClure'), ('Ahmed', 'Ali', 'M'), ('Alas', 'y', 'Argüelles'), ('Albert', 'Des', 'Méloizes'), ('Albert', 'E.', 'Ottinger'), ('Albert', 'Faton', 'de'), ('Albert', 'Geouffre', 'de'), ('Alberto', 'De', 'Mojana'), ('Alberto', 'Gómez', 'Jaramillo'), ('Alberto'

In [42]:
# attempt to change the 'NaN' names from DBpedia to name from BnF and the same for the reverse 
# (cf.https://stackoverflow.com/questions/29177498/python-pandas-replace-nan-in-one-column-with-value-from-corresponding-row-of-sec)
import pandas as pd
import numpy as np

def fx(x):
    if np.isnan(x['date_dbp']):
        return x['year_bnf ']
    else:
        return x['date_dbp']
result_test=result.apply(lambda x : fx(x),axis=1)

TypeError: ("ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''", 'occurred at index 0')

In [43]:
# Another method for trying to do it 
result.name_dbp.fillna(result.name_bnf, inplace=True)
del result['name_bnf']
result.columns = 'uri_dbp viaf name_dbp date_dbp uri_bnf sName year_bnf bio_bnf'.split()


ValueError: Length mismatch: Expected axis has 10 elements, new values have 8 elements

In [42]:
df1 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005', 'id006', 'id007'],
                    'first_name': ['Rivi', 'Wynnie', 'Kristos', 'Madalyn', 'Tobe', 'Regan', 'Kristin'],
                    'last_name': ['Valti', 'McMurty', 'Ivanets', 'Max', 'Riddich', 'Huyghe', 'Illis'],
                    'email': ['rvalti0@example.com', 'wmcmurty1@example.com', 'kivanets2@example.com',
                              'mmax3@example.com', 'triddich4@example.com', 'rhuyghe@example.com', 'killis4@example.com']
                    })

In [43]:
df2 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005'],
                    'image_url': ['http://example.com/img/id001.png', 'http://example.com/img/id002.jpg',
                                  'http://example.com/img/id003.bmp', 'http://example.com/img/id004.jpg',
                                  'http://example.com/img/id005.png']
                    })

In [11]:
df3_merged = pd.merge(df1, df2)
df3_merged 

,user_id,first_name,last_name,email,image_url
0,id001,Rivi,Valti,rvalti0@example.com,http://example.com/img/id001.png
1,id002,Wynnie,McMurty,wmcmurty1@example.com,http://example.com/img/id002.jpg
2,id003,Kristos,Ivanets,kivanets2@example.com,http://example.com/img/id003.bmp
3,id004,Madalyn,Max,mmax3@example.com,http://example.com/img/id004.jpg
4,id005,Tobe,Riddich,triddich4@example.com,http://example.com/img/id005.png
